In [42]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv
import openai
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load environment variables
load_dotenv()

# Get the API key from environment
api_key = os.getenv('API_KEY')
if not api_key:
    raise ValueError("API_KEY not found in .env file. Please add your OpenAI API key to the .env file.")

# Create OpenAI client
client = openai.OpenAI(api_key=api_key)

# Excel file path
EXCEL_FILE = os.path.join(os.path.dirname(os.path.abspath('__file__')), 'transactions.xlsx')

# Create Excel file if it doesn't exist
if not os.path.exists(EXCEL_FILE):
    # Create a new DataFrame with the required columns
    df = pd.DataFrame(columns=['Date', 'Description', 'Amount', 'Category', 'Type'])
    # Save it to Excel
    df.to_excel(EXCEL_FILE, index=False, sheet_name='Expenses')

## Excel File Structure

#The transactions are stored in a local Excel file named `transactions.xlsx` with the following columns:
1. Date (YYYY-MM-DD format)
2. Description (what the transaction was for)
3. Amount (positive number)
4. Category (e.g., Food, Transport, Salary, etc.)
5. Type (Income or Expense)

#The file is automatically created in the same folder as this notebook if it doesn't exist.

In [38]:
def parse_transaction(text):
    """Use OpenAI to parse transaction details from natural language input"""
    current_date = datetime.now().strftime('%Y-%m-%d')
    
    prompt = f"""
    Parse the following financial transaction(s) into a structured format. If multiple transactions are mentioned, return a list of JSON objects.
    Text: "{text}"
    
    Extract and return JSON with these fields for each transaction:
    - date: string in YYYY-MM-DD format (if no date is mentioned, use "{current_date}")
    - amount: number (positive)
    - description: string
    - category: string (e.g., Food, Transport, Salary, etc.)
    - type: string (either "Income" or "Expense")
    
    Examples:
    Input: "on July 20th paid $50 for internet"
    Output: {{"date": "2025-07-20", "amount": 50, "description": "internet", "category": "Utilities", "type": "Expense"}}
    
    Input: "$5 on groceries and $20 on lunch"
    Output: [
        {{"date": "{current_date}", "amount": 5, "description": "groceries", "category": "Food", "type": "Expense"}},
        {{"date": "{current_date}", "amount": 20, "description": "lunch", "category": "Food", "type": "Expense"}}
    ]
    
    Input: "yesterday spent $30 on gas and today $15 on coffee"
    Output: [
        {{"date": "2025-07-23", "amount": 30, "description": "gas", "category": "Transport", "type": "Expense"}},
        {{"date": "2025-07-24", "amount": 15, "description": "coffee", "category": "Food", "type": "Expense"}}
    ]
    """
    
    try:
        # Use the global client instance
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": f"You are a financial transaction parser. Respond only with valid JSON. For relative dates like 'yesterday' or 'last week', convert them to actual dates based on the current date being {current_date}. If multiple transactions are mentioned, return a list of JSON objects. If no date is mentioned, use {current_date}."},
                {"role": "user", "content": prompt}
            ]
        )
        
        parsed = json.loads(response.choices[0].message.content)
        # If single transaction is returned, convert to list
        if isinstance(parsed, dict):
            return [parsed]
        return parsed
    except Exception as e:
        print(f"Error calling OpenAI API: {str(e)}")
        return None

In [39]:
def update_excel(transaction_data):
    """Add a new transaction to the Excel file"""
    try:
        # Read existing data
        if os.path.exists(EXCEL_FILE):
            df = pd.read_excel(EXCEL_FILE, parse_dates=['Date'])  # Parse dates when reading
        else:
            df = pd.DataFrame(columns=['Date', 'Description', 'Amount', 'Category', 'Type'])
        
        # Create new row with date as datetime
        new_row = pd.DataFrame([{
            'Date': pd.to_datetime(transaction_data['date']),  # Convert to datetime
            'Description': transaction_data['description'],
            'Amount': transaction_data['amount'],
            'Category': transaction_data['category'],
            'Type': transaction_data['type']
        }])
        
        # Append new row and ensure all columns exist
        df = pd.concat([df, new_row], ignore_index=True)
        
        # Ensure Date column is datetime type and sort
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.sort_values('Date', ascending=True).reset_index(drop=True)
        
        # Convert dates to the desired format before saving
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        
        # Save back to Excel
        with pd.ExcelWriter(EXCEL_FILE, engine='openpyxl') as writer:
            df.to_excel(writer, index=False, sheet_name='Expenses')
            
        return True
    except Exception as e:
        print(f"Error updating Excel file: {e}")
        return False

In [ ]:
def create_transaction_interface():
    """Create an interactive interface for entering transactions"""
    # Create input widget
    text_input = widgets.Text(
        value='',
        placeholder='Enter transaction(s) (e.g., "$5 on coffee and $20 on lunch")',
        description='Transaction(s):',
        style={'description_width': 'initial'},
        layout={'width': '500px'}
    )
    
    # Create output widget for displaying results
    output = widgets.Output()
    
    def on_submit(b):
        with output:
            clear_output()
            if not text_input.value:
                print("Please enter a transaction.")
                return
                
            print(f"Processing: {text_input.value}")
            # Parse the transaction(s)
            transactions = parse_transaction(text_input.value)
            
            if not transactions:
                print("Could not parse the transaction(s). Please try again.")
                return
            
            # Display parsed data for confirmation
            print("\nParsed Transaction(s):")
            for i, transaction_data in enumerate(transactions, 1):
                print(f"\nTransaction {i}:")
                print(f"Date: {transaction_data['date']}")
                print(f"Description: {transaction_data['description']}")
                print(f"Amount: ${transaction_data['amount']}")
                print(f"Category: {transaction_data['category']}")
                print(f"Type: {transaction_data['type']}")
            
            # Update the Excel file with all transactions
            success = True
            for transaction_data in transactions:
                if not update_excel(transaction_data):
                    success = False
                    break
            
            if success:
                print(f"\nAll transactions successfully recorded in: {EXCEL_FILE}")
            else:
                print("\nError recording transactions. Please try again.")
            
            # Clear the input
            text_input.value = ''
    
    # Create submit button
    submit_button = widgets.Button(
        description='Submit',
        button_style='primary'
    )
    submit_button.on_click(on_submit)
    
    # Display the interface
    display(widgets.VBox([text_input, submit_button, output]))

# Create and display the interface
create_transaction_interface()